# Head files

In [1]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from net_cnn_lstm6 import MyNetwork
from torchnlp.word_to_vector import GloVe
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# Train the model

## Set arguments

In [ ]:
BATCH_SIZE = 1
C,H,W = 1,1,2400
# learn_rate = 0.0005
learn_rate=0.001
# num_epochs=50
num_epochs = 70

# ================================================ random
BATCH_SIZE = 1
C,H,W = 1,1,2400
# learn_rate = 0.0005
learn_rate=0.001
# num_epochs=50
num_epochs = 70

# ================================================ optimizer
import torch.optim as optim
device = torch.device("cuda:0")
gc.collect()
torch.cuda.empty_cache()
# ==损失函数权重
# ======== 二分类HC/DOC
# 计算总样本数量
# condition1
# total_samples = 887 + 985 + 879
# condition2
# total_samples = 929 + 1029 + 886
# condition3
# total_samples = 887 + 975 + 879
# rest
total_samples = 852 + 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 887, total_samples / (985 + 879)]
# condition2
# weights = [total_samples / 929, total_samples / (1029 + 886)]
# condition3
weights = [total_samples / 852, total_samples / (1051 + 872)]

# ======== 二分类MCS/UWS
# 计算总样本数量
# condition1
# total_samples = 985 + 879
# condition2
# total_samples = 1029 + 886
# condition3
# total_samples = 975 + 879
# rest
# total_samples = 1051 + 872
# 计算每个类别的权重
# condition1
# weights = [total_samples / 985, total_samples / 879]
# condition2
# weights = [total_samples / 1029, total_samples / 886]
# condition3
# weights = [total_samples / 975, total_samples / 879]
# condition3
# weights = [total_samples / 1051, total_samples / 872]
# 将权重转换为张量
weights_tensor = torch.tensor(weights, device=device)

# 定义交叉熵损失函数并设置权重
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
# criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

## Train

In [ ]:
def make_dataset(dataset,mode):
    # find the fold file
    count = 0
    for person in range(len(dataset)):
        filename = os.path.join(dataset.path, dataset.file_path[person])
        # extract the pure name of the file
        parts = filename.split("\\")
        file_name = parts[-1]
        name_without_extension = file_name.split(".")[0]
        # label or data
        file_last = name_without_extension.split("_")[-1]
        if file_last.isdigit(): # data
            # is this fold or not
            if int(file_last) == fold: # yes
                print(filename)
                count = count + 1
                data_map = torch.load(filename)
                # train or test
                if name_without_extension.split("_")[-2] == "train":
                    for i in range(data_map.size(0)):
                        train_data.append(data_map[i])
                elif name_without_extension.split("_")[-2] == "test":
                    for i in range(data_map.size(0)):
                        test_data.append(data_map[i])
                if count == 4:
                    del data_map
                    gc.collect()
                    torch.cuda.empty_cache() 
                    break
            else:   # not
                pass
        else: # label
            file_last = name_without_extension.split("_")[-4]
            file_mode = name_without_extension.split("_")[-2]
            if file_mode == mode.split("_")[-1]: # is this mode or not
                # is this fold or not
                if int(file_last) == fold: # yes
                    print(filename)
                    count = count + 1
                    data_map = torch.load(filename)
                    # train or test
                    if name_without_extension.split("_")[-5] == "train":
                        for i in range(data_map.size(0)):
                            train_label.append(data_map[i])
                    elif name_without_extension.split("_")[-5] == "test":
                        for i in range(data_map.size(0)):
                            test_label.append(data_map[i])
                    if count == 4:
                        del data_map
                        gc.collect()
                        torch.cuda.empty_cache() 
                        break
                else:   # not
                    pass
            else:
                pass
        del filename, parts, file_name, name_without_extension, file_last
        gc.collect()
        torch.cuda.empty_cache()    
# 定义LSTM超参数
input_size = 64  # 输入特征维度
hidden_size = 64  # 隐藏单元数量
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数量
# 创建模型实例
# model_list = ['', '_CNN', '_CNN_spa', '_CNN_spa_lstm']
# model_name = model_list[0]
# if model_name == model_list[0]: #  CascadeCept
#     from network_cnn_lstm import MyNetwork
# elif model_name == model_list[1]: #  CNN
#     from network_cnn_lstm_2 import MyNetwork
# elif model_name == model_list[2]: # CascadeCept_1
#     from network_cnn_lstm_3 import MyNetwork
# elif model_name == model_list[3]: # CascadeCept_2
#     from network_cnn_lstm_4 import MyNetwork
model_list = ['_1', '_2', '_3', '_4','_5','_6','_7','_8','_9','_10','_11']
model_name = model_list[5]
if model_name == model_list[0]: 
    from net_cnn_lstm1 import MyNetwork
elif model_name == model_list[1]: 
    from net_cnn_lstm2 import MyNetwork
elif model_name == model_list[2]: 
    from net_cnn_lstm3 import MyNetwork
elif model_name == model_list[3]: 
    from net_cnn_lstm4 import MyNetwork
elif model_name == model_list[4]: 
    from net_cnn_lstm5 import MyNetwork
elif model_name == model_list[5]: 
    from net_cnn_lstm6 import MyNetwork
elif model_name == model_list[6]: 
    from net_cnn_lstm7 import MyNetwork
elif model_name == model_list[7]: 
    from net_cnn_lstm8 import MyNetwork
elif model_name == model_list[8]: 
    from net_cnn_lstm9 import MyNetwork
elif model_name == model_list[9]: 
    from net_cnn_lstm10 import MyNetwork
elif model_name == model_list[10]: 
    from net_cnn_lstm11 import MyNetwork
model = MyNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)

# experimental dir: rest, conditionA, conditionB, conditionC
exper_dir = "rest59"
trial="_3"
# root_dir = f"../data/eegmap_split/{exper_dir}"
root_dir = f"../data/eegmap_split/{exper_dir}"
# classification = "hc_doc"/doc or "mcs_uws"/uws
classification = "hc_doc"
# classification = "mcs_uws"
fold_num = 5
best_val_loss = float('inf')
for fold in tqdm(range(5)):
    # train num folds
#     fold = 0 # 选择折数
    # -- prepare datasets
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    
    if classification == "hc_doc":
        # ---- hc
        dataset = MyData(root_dir, "train", "hc") # hc
        make_dataset(dataset,classification)
        dataset = MyData(root_dir, "test", "hc") # hc
        make_dataset(dataset,classification)
    # ---- mcs
    dataset = MyData(root_dir, "train", "mcs") # mcs
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "mcs") # hc
    make_dataset(dataset,classification)
    # ---- uws
    dataset = MyData(root_dir, "train", "uws") # uws
    make_dataset(dataset,classification)
    dataset = MyData(root_dir, "test", "uws") # hc
    make_dataset(dataset,classification)
    
    print(torch.stack(train_data).size())
    print(torch.stack(train_label).size())
    print(torch.stack(test_data).size())
    print(torch.stack(test_label).size())
    del dataset
    gc.collect()
    torch.cuda.empty_cache()  
    
    train_data = torch.stack(train_data)
    train_label = torch.stack(train_label)
    test_data = torch.stack(test_data)
    test_label = torch.stack(test_label)
    # train dataset
    train_td = TensorDataset(train_data, train_label)
    train_loader = DataLoader(train_td, batch_size = BATCH_SIZE, shuffle = True)
    # test dataset
    test_td = TensorDataset(test_data, test_label)
    test_loader = DataLoader(test_td, batch_size = BATCH_SIZE, shuffle = True)
    del train_data, train_label, test_data, test_label, train_td, test_td
    gc.collect()
    torch.cuda.empty_cache()
    
    # set model for each fold
    model = MyNetwork(input_size, hidden_size, num_layers, output_size)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr = learn_rate)
#     scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30, 50], gamma=0.2)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    # -- start training
    start_time = time.time()
    # train and test step records
    total_train_step = 0
    total_test_step = 0
    min_test_loss = 1000
    # add Tensorboard
#     writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_train_{fold}")
#     writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_{fold}")
#     writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}/logs_test_acc_{fold}")
    writer_train = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_train_{fold}")
    writer_valid = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_test_{fold}")
    writer_valid_acc = SummaryWriter(f"../logs/{classification}/{exper_dir}{model_name}{trial}/logs_test_acc_{fold}")
    for i in tqdm(range(num_epochs)):  
        print(f"========= Epoch {i} Training =========")
        # train steps
        model.train()
        for data in train_loader:
            # x, y
            data_map, label=data
#             data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
            data_map_reshaped = torch.reshape(data_map, (59, 1, 1, 2400))
            label_int = label.long()
            data_map_reshaped=data_map_reshaped.to(device)
            label_int=label_int.to(device)
            del data_map, label
            gc.collect()
            torch.cuda.empty_cache()
            # y_pred
            label_pred = model(data_map_reshaped)
            # Loss Computation and Optimization
            loss = criterion(label_pred,label_int)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # draw tensorboard
            total_train_step = total_train_step + 1
            # print info
            if total_train_step % 500 == 0:
                end_time = time.time()
                print(label_pred)
                print(f"Train time: {end_time - start_time}")
                print(f"Train steps: {total_train_step}, Loss: {loss.item()}")
            writer_train.add_scalar("train_loss",loss.item(),total_train_step)
            # Clear gpu
            del data, data_map_reshaped, label_int, label_pred, loss
            gc.collect()
            torch.cuda.empty_cache()
        
        # Evaluation and save the best model
        print(f"========= Epoch {i} Testing =========")
        model.eval()
        test_loss = 0
        total_test_loss = 0
        test_count = 0
        total_test_acc = 0
        with torch.no_grad():
            for data in test_loader:
                test_count = test_count + 1
                # x, y
                data_map, label=data
#                 data_map_reshaped = torch.reshape(data_map, (110, 1, 1, 2400))
                data_map_reshaped = torch.reshape(data_map, (59, 1, 1, 2400))
                label_int = label.long()
                data_map_reshaped = data_map_reshaped.to(device)
                label_int = label_int.to(device)
                del data_map, label
                gc.collect()
                torch.cuda.empty_cache()
                # y_pred
                label_pred_test = model(data_map_reshaped)
                loss = criterion(label_pred_test,label_int)
#                 print(label_pred_test)
                # accuracy 
                total_test_acc = total_test_acc + ((label_pred_test.argmax(1)) == label_int).sum()
                # change lr
                test_loss += loss.item()
                test_loss /= len(test_loader)
                if test_loss < best_val_loss:
                    best_val_loss = test_loss
                scheduler.step(test_loss)
                # draw tensorboad
                total_test_loss = total_test_loss + loss
                if test_count % 100 == 0:
                    print(f"Loss: {total_test_loss} Accuracy: {total_test_acc/test_count}")
                # Clear gpu
                del data_map_reshaped, label_int, label_pred_test, loss, data
                gc.collect()
                torch.cuda.empty_cache()
        print(f"Total Loss: {total_test_loss} Total Accuracy: {total_test_acc/test_count}")
        writer_valid.add_scalar("test_loss", total_test_loss, total_test_step)
        writer_valid_acc.add_scalar("test_acc", total_test_acc/test_count, total_test_step)
        total_test_step = total_test_step + 1
        print("..........Saving the model..........")
#         torch.save(model.state_dict(),f"../models/{classification}/{exper_dir}{model_name}/Fold{fold}_Epoch{i}.pt") 
        torch.save(model.state_dict(),f"../models/{classification}/{exper_dir}{model_name}{trial}/Fold{fold}_Epoch{i}.pt") 
#         if total_test_loss < min_test_loss:
#             min_test_loss = total_test_loss
#             print("..........Saving the model..........")
#             torch.save(model.state_dict(),f"../model/{exper_dir}/Fold{fold}_Epoch{i}.pt")